# SQL

## Цели исследования

Наша компания купила крупный сервис для чтения книг по подписке.<br>
Необходимо проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Знакомство с данными

In [1]:
# импортируем библиотеки 
import pandas as pd 
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

Выведем первые 5 строк таблиц и познакомимся с данными.

### Таблица books

Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [3]:
query = '''SELECT * FROM books LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица authors

Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

In [4]:
query = '''SELECT * FROM authors LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица publishers

Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица ratings

Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [6]:
query = '''SELECT * FROM ratings LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица reviews

Содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [7]:
query = '''SELECT * FROM reviews LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Решение задач

Выполним следующие задания:
1. Посчитаем, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитаем количество обзоров и среднюю оценку;
3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
4. Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками;
5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

### Книги, которые вышли после 1 января 2000 года

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [8]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date >= '2000-01-01'
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,821


После 1 января 2000 года (включая 1 января) вышла 821 книга.

### Количество обзоров и средняя оценка

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [9]:
query = '''
SELECT b.book_id,
    b.title,
    COUNT(DISTINCT rv.review_id) count_review,
    ROUND(AVG(rt.rating), 2) avg_rating
FROM books b
LEFT JOIN reviews rv ON rv.book_id = b.book_id
LEFT JOIN ratings rt ON rt.book_id = b.book_id
GROUP BY b.book_id;
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count_review,avg_rating
0,1,'Salem's Lot,2,3.67
1,2,1 000 Places to See Before You Die,1,2.50
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,4,1491: New Revelations of the Americas Before C...,2,4.50
4,5,1776,4,4.00
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,997,Xenocide (Ender's Saga #3),3,3.40
997,998,Year of Wonders,4,3.20
998,999,You Suck (A Love Story #2),2,4.50


Для каждой книги посчитали количество обзоров и среднюю оценку.

### Издательство, которое выпустило наибольшее число книг

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры.

In [10]:
query = '''
SELECT p.publisher_id,
    p.publisher,
    COUNT(b.book_id) count_books
FROM publishers p
LEFT JOIN books b ON b.publisher_id=p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY count_books DESC
LIMIT 1;
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,count_books
0,212,Penguin Books,42


Издательством, которое выпустило наибольшее число книг толще 50 страниц, оказалось «Penguin Books» - оно выпустило 42 книги.

### Автор с самой высокой средней оценкой книг

Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками.

In [11]:
query = '''
SELECT a.author_id,
    a.author,
    ROUND(AVG(rt.rating), 2) avg_rating
FROM authors a
LEFT JOIN books b ON b.author_id=a.author_id
LEFT JOIN ratings rt ON rt.book_id=b.book_id
WHERE b.book_id IN (SELECT b.book_id
                    FROM books b
                    JOIN ratings rt ON rt.book_id=b.book_id
                    GROUP BY b.book_id
                    HAVING COUNT(rt.rating_id) >= 50)
GROUP BY a.author_id
ORDER BY avg_rating DESC
LIMIT 1;
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.29


Автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré (средняя оценка книг 4,29).

### Cреднее количество обзоров от пользователей

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = '''
WITH 
k AS (
SELECT username,
    COUNT(review_id) count_review
FROM reviews 
WHERE username IN (SELECT username
                    FROM ratings 
                    GROUP BY username
                    HAVING COUNT(rating_id) > 48)
                    GROUP BY username)
SELECT AVG(count_review) avg_count_review
FROM k
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,avg_count_review
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24.

## Общий вывод по результатам решения задач

- После 1 января 2000 года (включая 1 января) вышла 821 книга;
- Для каждой книги посчитали количество обзоров и среднюю оценку;
- Издательством, которое выпустило наибольшее число книг толще 50 страниц, оказалось «Penguin Books» - оно выпустило 42 книги;
- Автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré (средняя оценка книг 4,29);
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24.